In [5]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="Rj7J0lDDcL5F3UabGv9S")
    project = rf.workspace("brain-o889o").project("brain-qicsj")
    version = project.version(1)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = YOLO("yolov8n")  # Adjust model if needed




Dataset already exists. Using the existing dataset.


In [6]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "valid", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(4):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Scanning: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/valid/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/test/labels

Number of images per class:
Class 0: 313 images
Class 1: 322 images
Class 2: 351 images
Class 3: 346 images


In [7]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="Brain org v8",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=64,  # Adjust batch size for better GPU utilization
    workers=16,  # Use multiple data loading workers
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=Brain org v82, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_j

100%|██████████| 5.35M/5.35M [00:04<00:00, 1.16MB/s]


AMP: checks passed ✅


train: Scanning /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/train/labels... 5265 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5265/5265 [00:07<00:00, 668.21it/s]


train: New cache created: /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/train/labels.cache


...

                   all       1000       1000      0.968       0.97      0.988        0.9

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.62G     0.5413     0.4081     0.9568          5        640: 100%|██████████| 83/83 [00:17<00:00,  4.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:12<00:00,  2.65it/s]


                   all       1000       1000      0.969      0.968      0.989      0.901

100 epochs completed in 0.794 hours.
Optimizer stripped from runs/detect/Brain org v82/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/Brain org v82/weights/best.pt, 6.2MB

Validating runs/detect/Brain org v82/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:07<00:00,  4.40it/s]


                   all       1000       1000       0.97      0.969      0.988      0.904
                     0        228        228      0.995      0.987      0.995      0.921
                     1        256        256       0.93      0.957      0.984      0.894
                     2        252        252      0.965       0.94      0.976      0.876
                     3        264        264      0.989      0.992      0.995      0.925
Speed: 0.2ms preprocess, 1.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/Brain org v82


In [8]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Brain/Brain Org/valid/labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [00:10<00:00,  1.53it/s]


                   all       1000       1000      0.969      0.969      0.988      0.904
                     0        228        228      0.995      0.987      0.995      0.923
                     1        256        256       0.93      0.957      0.984      0.894
                     2        252        252      0.964       0.94      0.976      0.877
                     3        264        264      0.989      0.992      0.995      0.924
Speed: 0.3ms preprocess, 2.5ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/Brain org v822
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd0d3ce1d00>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)